<a href="https://colab.research.google.com/github/username-usr/makingbrandnewsecretkey/blob/main/openlabs_2_%5Bfinding_entropy_in_different_stage%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# goal is to get entropy before and after processing, i.e, pre vs post processing

In [1]:
!pip install antropy

In [2]:
import pandas as pd
import numpy as np
import antropy as ant
import glob
import os

# 1. Setup paths
folder_path = '/content'
files = sorted(glob.glob(os.path.join(folder_path, "s*.csv")))

results = []

for file in files:
    df = pd.read_csv(file, header=None)
    df.columns = [f'Ch_{i}' for i in range(df.shape[1])]

    subject_name = os.path.basename(file)
    subject_entropies = {'Subject': subject_name}

    # 2. Loop through each column (channel)
    for col in df.columns:
        signal = df[col].values

        # Calculate Permutation Entropy
        # normalize=True gives a result between 0 and 1
        pe = ant.perm_entropy(signal, order=3, delay=1, normalize=True)
        subject_entropies[col] = pe

    results.append(subject_entropies)

# 3. Create a summary Table
entropy_df = pd.DataFrame(results)
print("Quality Assessment (Permutation Entropy):")
print(entropy_df.head())

Quality Assessment (Permutation Entropy):
   Subject      Ch_0      Ch_1      Ch_2      Ch_3      Ch_4      Ch_5  \
0  s00.csv  0.594520  0.594201  0.580307  0.577425  0.598075  0.603040   
1  s01.csv  0.629654  0.622993  0.608168  0.605340  0.618310  0.616409   
2  s02.csv  0.657624  0.642175  0.632246  0.636037  0.659414  0.646649   
3  s03.csv  0.603315  0.607620  0.580404  0.576402  0.587866  0.588967   
4  s04.csv  0.587623  0.582907  0.578741  0.575761  0.593314  0.582109   

       Ch_6      Ch_7      Ch_8      Ch_9     Ch_10     Ch_11     Ch_12  \
0  0.586967  0.579534  0.581589  0.577251  0.585544  0.574650  0.583056   
1  0.632520  0.611319  0.602684  0.603119  0.604715  0.612529  0.600899   
2  0.651780  0.641411  0.623138  0.616211  0.656080  0.654479  0.612998   
3  0.587152  0.585232  0.571733  0.567545  0.590046  0.568431  0.574302   
4  0.582021  0.584174  0.574788  0.578748  0.586159  0.571969  0.567052   

      Ch_13     Ch_14     Ch_15     Ch_16     Ch_17     Ch_18 